In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

import requests
from bs4 import BeautifulSoup
import re

# Preamble — Prepare to fetch data

These are the root URLs for the two kinds of requests we need to perform: filters and lists.

First, the filter dialog is used to

1. Retreive the IDs for the years and semesters, as well as for the _Informatique_ field of study, we care about
2. Look for the ID of the associated list.

Lists contain the actual data we care about.

Both types of request use a fixed `ww_i_reportModel` constant, which we assume is used for versioning.

In [5]:
root_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.'
report = '&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&'
filter_path = root_url + 'filter?ww_b_list=1' + report
list_path = root_url + 'html?' + report
[ filter_path, list_path ]

['http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&',
 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&']

Next, we look up the IDs for the years and semesters we're interested in.

In [6]:
# Find the IDs for the individual years

req = requests.get(filter_path)

if req.status_code != 200:
  raise Exception('Request failed')

parser = BeautifulSoup(req.text, "html.parser")

years = parser.find('select', attrs={"name": "ww_x_PERIODE_ACAD"}).findAll('option')
year_ids = {year.text: year['value'] for year in years}

year_ids

{'': 'null',
 '2007-2008': '978181',
 '2008-2009': '978187',
 '2009-2010': '978195',
 '2010-2011': '39486325',
 '2011-2012': '123455150',
 '2012-2013': '123456101',
 '2013-2014': '213637754',
 '2014-2015': '213637922',
 '2015-2016': '213638028',
 '2016-2017': '355925344'}

In [7]:
informatique_id = parser.find(
    'select',
    attrs={"name": "ww_x_UNITE_ACAD"}
).find(
    (lambda tag: tag.text == 'Informatique')
)['value']

informatique_id

'249847'

In [8]:
periods = parser.find('select', attrs={'name': 'ww_x_PERIODE_PEDAGO'})
period_ids = {}

for el in periods:
    period_ids[el.text] = el['value']

period_ids

{'': 'null',
 'Bachelor semestre 1': '249108',
 'Bachelor semestre 2': '249114',
 'Bachelor semestre 3': '942155',
 'Bachelor semestre 4': '942163',
 'Bachelor semestre 5': '942120',
 'Bachelor semestre 5b': '2226768',
 'Bachelor semestre 6': '942175',
 'Bachelor semestre 6b': '2226785',
 'Master semestre 1': '2230106',
 'Master semestre 2': '942192',
 'Master semestre 3': '2230128',
 'Master semestre 4': '2230140',
 'Mineur semestre 1': '2335667',
 'Mineur semestre 2': '2335676',
 'Mise à niveau': '2063602308',
 'Projet Master automne': '249127',
 'Projet Master printemps': '3781783',
 'Semestre automne': '953159',
 'Semestre printemps': '2754553',
 'Stage automne 3ème année': '953137',
 'Stage automne 4ème année': '2226616',
 'Stage printemps 3ème année': '983606',
 'Stage printemps 4ème année': '2226626',
 'Stage printemps master': '2227132'}

In [9]:
def get_gps_id(period, year):
    query = 'ww_x_PERIODE_PEDAGO=' + period_ids[period] + '&ww_x_PERIODE_ACAD=' + year_ids[year]
    req = requests.get(filter_path + 'ww_x_UNITE_ACAD=' + informatique_id + '&' + query)
    
    
    if req.status_code != 200:
      raise Exception('Request failed')
    
    parser = BeautifulSoup(req.text, "html.parser")

    onclick_value = parser.findAll('a', class_ = 'ww_x_GPS')[1]['onclick']
    
    return re.search('ww_x_GPS=(\\d+)', onclick_value).group(1)

get_gps_id(period='Bachelor semestre 1', year='2016-2017')

'2021043255'

In [10]:
def get_table(period, year):
    query = 'ww_x_GPS=' + get_gps_id(period, year) + '&ww_x_UNITE_ACAD=' + informatique_id
    req = requests.get(list_path + query)

    if req.status_code != 200:
      raise Exception('Request failed')

    parser = BeautifulSoup(req.text, "html.parser")

    return list(map((lambda x: [el.text for el in x.findAll('td')]), parser.findAll('tr')[2:]))

get_table(period='Bachelor semestre 6', year='2008-2009')[0]

['Monsieur',
 'Balas\xa0Martin',
 '',
 '',
 '',
 '',
 '',
 'Présent',
 '',
 '',
 '174774',
 '']

# Task 1

> Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [132]:
result_dict = {} 
attente = {}
conge = {}
for year in range(2007, 2017):
    for sem in range(1, 7):
        table = get_table(period='Bachelor semestre ' + str(sem), year=str(year) + '-' + str(year + 1))
        for record in table:
            statut = record[7]
            id = int(record[10])  # will raise expcetion if Noone or empty string 
            if  statut == 'Présent':                               
                gender = record[0] 
                if gender not in ['Monsieur', 'Madame']:
                    raise Exception('Unexpected gender:' + gender + '; sem: ' + str(sem) + ' year: ' + str(year) + ' id: ' + str(id))
                gender = int(gender == 'Monsieur')  # => 1 for Monsieur, 0 for Madame
                if id not in result_dict:
                    if sem == 1:  # we consider only students who started studying in 2007 or later
                        result_dict[id] = {'gender': gender, 'sem_count': 0, 'sems': []}
                    else:
                        continue
                else:
                    if result_dict[id]['gender'] != gender:
                        raise Expception('Inconsistent gender!')
                result_dict[id]['sem_count'] += 1
                result_dict[id]['sems'].append(str(sem) + '_' + str(year))
            elif statut == 'Congé':
                if id not in conge:
                    conge[id] = {'c_sems': [], 'c_sem_count': 0}
                conge[id]['c_sems'].append(str(sem) + '_' + str(year))
                conge[id]['c_sem_count'] += 1     
            elif statut == 'Attente':
                if id not in attente:
                    attente[id] = {'a_sems': [], 'a_sem_count': 0}
                attente[id]['a_sems'].append(str(sem) + '_' + str(year))
                attente[id]['a_sem_count'] += 1                
            else: raise Exception('Unexpected statut: ' + statut + '; sem: ' + str(sem) + ' year: ' + str(year) + ' id: ' + str(id))

In above alpgorithm we load all the students from desired period (2007+). However, for the sake of simplicity we are only interested in students who started at 2007 or later. If we were to consider students who started earlier we would have to make lots of assumptions which would result in making up data. 

We also check for the statut of the student at given semester. We have found 3 different types of statutes - Présent, Congé and Attente. Later on we will try to analyze their meaning and contribution.

When it comes to Période pédagogique we take into account only Bachelor Semester 1 - 6. We have decided to ignore Semestre automne and Semestre printemps as they contain only exchange students. Exchange studants are staying only one semster or two and so using them in our computation would skew the averages to artifical low numbers. We have also ignored Mise à niveau, Bachelor Semestre 5b, Bachelor Semestre 6b because they are empty for Unité académique == Informatique. All the other Périodes pédagogiques (Master semestre N, Mineur semestre N, Projet Master automne/printemps) are ignored for this part of exercise because they regard master students.     

We also disregard Type de semestre becuase it does not bring us any new information. As a rule of a thumb Bachelor semestres 1, 3 and 5 are always Semestre d'automne while Bachelor semestres 2, 4 and 6 are always Semestre de printemps. Combination of Bachelor Semestre N with different season (e.g. Bachelor semestre 1 and Semestre de printemps) results always in empty result. 

In [ ]:
to_delete = []
for key, value in result_dict.items():
    if not any(sem.startswith('6') for sem in value['sems']):
        to_delete.append(key)
for key in to_delete:
    del result_dict[key]

In above code we check if student has an entry for semester 6 in his record. If he has we assume he sucessfully finished his studies, even though technically the fact that he took part in 6th semester does not mean he has finished it sucessfully. Then we remove all the students who don't have the entry for semester 6 in their semester list as they haven't finished the studies yet becuase they are not suitable for the computation. 

In [141]:
len(result_dict)

297

By the removal of irrelevant data we decreased size of our dataset from 1476 to 297 samples.

In [135]:
def intersect_dicts(dict1, dict2):
    keys_intersection = set(dict1.keys()).intersection(set(dict2.keys()))
    res = {}
    for key in keys_intersection:
        res[key] = {**dict1[key], **dict2[key]}
    return res

In [143]:
attente_int_result = set(attente.keys()).intersection(set(result_dict.keys()))
len(attente_int_result)

1

In [144]:
intersect_dicts(result_dict, attente)

{223892: {'a_sem_count': 2,
  'a_sems': ['1_2014', '2_2014'],
  'gender': 1,
  'sem_count': 9,
  'sems': ['1_2012',
   '2_2012',
   '1_2013',
   '2_2013',
   '3_2014',
   '4_2014',
   '5_2015',
   '6_2015',
   '5_2016']}}

Attente statut has occured only in 1 case out of 297. It is overlapping with semesters where student had statut Présent. We consider the statut Présent to be of higher importance and thus we disregard the Attente statut.  

In [151]:
conge_int_result = set(conge.keys()).intersection(set(result_dict.keys()))
len(conge_int_result)

24

In [152]:
intersect_dicts(result_dict, conge)

{169731: {'c_sem_count': 1,
  'c_sems': ['5_2010'],
  'gender': 1,
  'sem_count': 7,
  'sems': ['1_2007',
   '2_2007',
   '3_2008',
   '4_2008',
   '3_2009',
   '4_2009',
   '6_2010']},
 174905: {'c_sem_count': 2,
  'c_sems': ['3_2008', '4_2008'],
  'gender': 1,
  'sem_count': 8,
  'sems': ['1_2007',
   '2_2007',
   '3_2009',
   '4_2009',
   '5_2010',
   '6_2010',
   '5_2011',
   '6_2011']},
 175379: {'c_sem_count': 2,
  'c_sems': ['1_2007', '2_2007'],
  'gender': 1,
  'sem_count': 6,
  'sems': ['1_2008', '2_2008', '3_2009', '4_2009', '5_2010', '6_2010']},
 181244: {'c_sem_count': 2,
  'c_sems': ['1_2008', '2_2008'],
  'gender': 1,
  'sem_count': 10,
  'sems': ['1_2007',
   '2_2007',
   '1_2009',
   '2_2009',
   '3_2010',
   '4_2010',
   '5_2011',
   '6_2011',
   '5_2012',
   '6_2012']},
 184372: {'c_sem_count': 2,
  'c_sems': ['5_2011', '6_2011'],
  'gender': 1,
  'sem_count': 10,
  'sems': ['1_2008',
   '2_2008',
   '1_2009',
   '2_2009',
   '3_2010',
   '4_2010',
   '5_2012',
   '6_

In this step we have analyzed Congé statut. We have 24 students who have recorded at least one semester with htis statut. By reading through data we see that semsters with statut Congé are mutually exclusive with semsesters with statut Présent. Congé in translatio nto English means Leave. Thus we assume that during these semesters the student went eithe rfor exchange studies or had a break in the studies due to, for example, health or family reasons. 

We consider this to be valid part of our data and therefore we have decided to do two calculations - one will take into account only semesters when the student was physically present and the other will consider also semesters when the student was on leave.

In [153]:
tmp_dict = intersect_dicts(result_dict, attente)
intersect_dicts(tmp_dict, conge)

{}

There are 0 students in our filtered dataset who have both statut Congé and Attente. 

In [157]:
no_of_males = sum([val['gender'] for val in result_dict.values()])
no_of_males

273

In [159]:
no_of_females = len(result_dict) - no_of_males
no_of_females

24

As we didn't get any exception regarding gender while parsing the data we can safely assume that there was no gender value missing.

Thus len(result_dict) - no_of_males is a valid computation.

In [184]:
male_average_wo_leave = sum([val['sem_count'] for val in result_dict.values() if val['gender'] == 1]) / no_of_males 
print('Average number of semesters to finish bachelor studies without leave for male student is ' + "{0:.2f}".format(male_average_wo_leave))
female_average_wo_leave = sum([val['sem_count'] for val in result_dict.values() if val['gender'] == 0]) / no_of_females
print('Average number of semesters to finish bachelor studies without leave for female student is ' + "{0:.2f}".format(female_average_wo_leave))

print('\n')

male_average_w_leave = sum([val['sem_count'] + conge.get(key, {}).get('c_sem_count', 0) for  key, val in result_dict.items() if val['gender'] == 1]) / no_of_males 
print('Average number of semesters to finish bachelor studies including leave for male student is ' + "{0:.2f}".format(male_average_w_leave))
female_average_w_leave = sum([val['sem_count'] + conge.get(key, {}).get('c_sem_count', 0) for key, val in result_dict.items() if val['gender'] == 0]) / no_of_females
print('Average number of semesters to finish bachelor studies including leave for female student is ' + "{0:.2f}".format(female_average_w_leave))

Average number of semesters to finish bachelor studies without leave for male student is 7.33
Average number of semesters to finish bachelor studies without leave for female student is 6.75


Average number of semesters to finish bachelor studies including leave for male student is 7.44
Average number of semesters to finish bachelor studies including leave for female student is 6.96


TODO check statistical significance

# Task 2

> Perform a similar operation to what described above, this time for Master students. Notice that this data is more tricky, as there are many missing records in the IS-Academia database. Therefore, try to guess how much time a master student spent at EPFL by at least checking the distance in months between Master semestre 1 and Master semestre 2. If the Mineur field is not empty, the student should also appear registered in Master semestre 3. Last but not the least, don't forget to check if the student has an entry also in the Projet Master tables. Once you can handle well this data, compute the "average stay at EPFL" for master students. Now extract all the students with a Spécialisation and compute the "average stay" per each category of that attribute -- compared to the general average, can you find any specialization for which the difference in average is statistically significant?

In [205]:
result_dict = {} 
attente = {}
conge = {}
periods = []

for sem in range(1, 4):
    periods.append(('Master semestre ' + str(sem), str(sem)))                    
periods.append(('Projet Master automne', 'pa'))
periods.append(('Projet Master printemps', 'pp'))

for year in range(2007, 2017):
    for period_name, period_code in periods:
        table = get_table(period=period_name, year=str(year) + '-' + str(year + 1))
        for record in table:
            statut = record[7]
            id = int(record[10])  # will raise expcetion if Noone or empty string 
            if  statut in ['Présent', 'Stage']:                               
                gender = record[0] 
                if gender not in ['Monsieur', 'Madame']:
                    raise Exception('Unexpected gender:' + gender + '; period: ' + str(period_name) + ' year: ' + str(year) + ' id: ' + str(id))
                gender = int(gender == 'Monsieur')  # => 1 for Monsieur, 0 for Madame
                spec = record[4]
                if id not in result_dict:
#                     if period == 1:  # we consider only students who started studying in 2007 or later
                    result_dict[id] = {'gender': gender, 'spec': spec, 'sem_count': 0, 'sems': []}
#                     else:
#                         continue
                else:
                    if result_dict[id]['gender'] != gender:
                        raise Expception('Inconsistent gender!')
#                     if result_dict[id]['spec'] == '':
                    result_dict[id]['spec'] = spec
#                     elif result_dict[id]['spec'] != spec:
#                         raise Exception('Inconsistent specialization! spec_prev: ' + result_dict[id]['spec'] + ' spec_cur: ' + spec)
                result_dict[id]['sem_count'] += 1
                stage_sign = 's' if statut == 'Stage' else ''
                result_dict[id]['sems'].append(str(period_code) + stage_sign + '_' + str(year))
            elif statut == 'Congé':
                if id not in conge:
                    conge[id] = {'c_sems': [], 'c_sem_count': 0}
                conge[id]['c_sems'].append(str(period_code) + '_' + str(year))
                conge[id]['c_sem_count'] += 1     
            elif statut == 'Attente':
                if id not in attente:
                    attente[id] = {'a_sems': [], 'a_sem_count': 0}
                attente[id]['a_sems'].append(str(period_code) + '_' + str(year))
                attente[id]['a_sem_count'] += 1                
            else: raise Exception('Unexpected statut: ' + statut + '; period: ' + str(period_name) + ' year: ' + str(year) + ' id: ' + str(id))

In [206]:
len(result_dict)

968

In [207]:
missing_sem_1 = {key: val for key, val in result_dict.items() if not any(sem.startswith('1') for sem in val['sems'])}
len(missing_sem_1)

84

In [208]:
missing_sem_1

{128911: {'gender': 1,
  'sem_count': 2,
  'sems': ['2_2007', '3_2007'],
  'spec': 'Internet computing'},
 129093: {'gender': 1,
  'sem_count': 2,
  'sems': ['2_2007', '3s_2007'],
  'spec': ''},
 129326: {'gender': 1,
  'sem_count': 2,
  'sems': ['2_2007', '3_2007'],
  'spec': ''},
 145546: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 145957: {'gender': 1,
  'sem_count': 2,
  'sems': ['3_2007', 'pps_2007'],
  'spec': ''},
 146441: {'gender': 0, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 146752: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 147358: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 147362: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 147366: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 147445: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 148900: {'gender': 1, 'sem_count': 1, 'sems': ['3_2007'], 'spec': ''},
 152238: {'gender': 1,
  'sem_count': 1,
  'sems': 

In [210]:
missing_projet = {key: val for key, val in result_dict.items() if not any(sem.startswith('p') for sem in val['sems'])}
len(missing_projet)

853

In [211]:
len(result_dict)

968

In [216]:
178942 in conge

True

In [213]:
len(conge)

49

In [214]:
conge

{146742: {'c_sem_count': 1, 'c_sems': ['pp_2010']},
 153324: {'c_sem_count': 1, 'c_sems': ['3_2007']},
 154080: {'c_sem_count': 2, 'c_sems': ['1_2007', '2_2007']},
 159852: {'c_sem_count': 2, 'c_sems': ['pp_2007', 'pa_2008']},
 160213: {'c_sem_count': 1, 'c_sems': ['pp_2009']},
 161127: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 165819: {'c_sem_count': 1, 'c_sems': ['pp_2008']},
 166124: {'c_sem_count': 1, 'c_sems': ['3_2008']},
 166260: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 166344: {'c_sem_count': 1, 'c_sems': ['pp_2010']},
 166491: {'c_sem_count': 4,
  'c_sems': ['2_2007', '2_2008', '1_2009', 'pp_2010']},
 166748: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 167439: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 167530: {'c_sem_count': 2, 'c_sems': ['1_2009', '2_2009']},
 170235: {'c_sem_count': 1, 'c_sems': ['pp_2011']},
 170242: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 170414: {'c_sem_count': 2, 'c_sems': ['1_2008', '2_2008']},
 17

# Bonus-Task

> BONUS: perform the gender-based study also on the Master students, as explained in 1. Use scatterplots to visually identify changes over time. Plot males and females with different colors -- can you spot different trends that match the results of your statistical tests?